In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [14]:
!echo $JAVA_HOME

In [15]:
from platform import python_version

print(python_version())

import pandas as pd
import numpy as np


print(pd.__version__)

print(np.__version__)


3.6.3
0.19.2
1.17.0


In [16]:
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("reading csv") \
        .getOrCreate()
    
data_file = '/opt/app-root/src/data/train.psv'
sdfData = scSpark.read.csv(data_file, header=True, sep="|").cache()
print('Total Records = {}'.format(sdfData.count()))
sdfData.show()

Total Records = 1168686
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|    PID| HR|O2Sat| Temp|SBP|  MAP|DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2|  pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI| Hct| Hgb|PTT|WBC|Fibrinogen|Platelets|  Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|p000001|NaN|

In [17]:
sdfData.registerTempTable("train")
output =  scSpark.sql('SELECT count(*) from train')
output.show()

+--------+
|count(1)|
+--------+
| 1168686|
+--------+



In [28]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages mysql:mysql-connector-java:jar:8.0.21 pyspark-shell'

output = scSpark.sql('SELECT * from train')
output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://10.128.1.138/sepsis',
    driver='com.mysql.jdbc.Driver',
    #driver='com.mysql.cj.jdbc.Driver',
    #driver='mysql-connector-java.Driver',
    #driver='mysql-connector-python',
    dbtable='train',
    user='root',
    password='Sepsis_Parkar_2020').mode('append').save()

+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|    PID| HR|O2Sat| Temp|SBP|  MAP|DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2|  pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI| Hct| Hgb|PTT|WBC|Fibrinogen|Platelets|  Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|p000001|NaN|  NaN|  NaN|NaN|  NaN|Na

Py4JJavaError: An error occurred while calling o338.save.
: com.mysql.cj.jdbc.exceptions.CommunicationsException: Communications link failure

The last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server.
	at com.mysql.cj.jdbc.exceptions.SQLError.createCommunicationsException(SQLError.java:174)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:64)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:836)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:456)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:246)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:197)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverWrapper.connect(DriverWrapper.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:61)
	at org.apache

In [19]:
data_file1 = '/opt/app-root/src/data/test.psv'
sdfData1 = scSpark.read.csv(data_file1, header=True, sep="|").cache()
print('Total Records = {}'.format(sdfData1.count()))
sdfData.show()

Total Records = 190615
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|    PID| HR|O2Sat| Temp|SBP|  MAP|DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2|  pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI| Hct| Hgb|PTT|WBC|Fibrinogen|Platelets|  Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+---+-----+-----+---+-----+---+----+-----+----------+----+----+----+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+----+---+---+----------+---------+-----+------+-----+-----+-----------+------+-----------+
|p000001|NaN| 

In [20]:
sdfData1.registerTempTable("test")
output =  scSpark.sql('SELECT count(*) from test')
output.show()

+--------+
|count(1)|
+--------+
|  190615|
+--------+



In [21]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:/opt/app-root/lib/python3.6/site-packages/pyspark/jars/mysql-connector-java-8.0.21.jar pyspark-shell'

output = scSpark.sql('SELECT * from test')
output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://10.128.1.138/sepsis',
    #driver='com.mysql.jdbc.Driver',
    driver='com.mysql.cj.jdbc.Driver',
    #driver='mysql-connector-java.Driver',
    #driver='mysql-connector-python',
    dbtable='test',
    user='root',
    password='Sepsis_Parkar_2020').mode('append').save()

+-------+---+-----+----+-----+-----+----+----+-----+----------+----+----+---+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+---+---+----+----------+---------+---+------+-----+-----+-----------+------+-----------+
|    PID| HR|O2Sat|Temp|  SBP|  MAP| DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2| pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI| Hct|Hgb|PTT| WBC|Fibrinogen|Platelets|Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+---+-----+----+-----+-----+----+----+-----+----------+----+----+---+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+----+---+---+----+----------+---------+---+------+-----+-----+-----------+------+-----------+
|p115001|NaN|  NaN| NaN|  NaN|  NaN| NaN

Py4JJavaError: An error occurred while calling o105.save.
: com.mysql.cj.jdbc.exceptions.CommunicationsException: Communications link failure

The last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server.
	at com.mysql.cj.jdbc.exceptions.SQLError.createCommunicationsException(SQLError.java:174)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:64)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:836)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:456)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:246)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:197)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:61)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils

In [22]:
data_file2 = '/opt/app-root/src/data/validation.psv'
sdfData2 = scSpark.read.csv(data_file2, header=True, sep="|").cache()
print('Total Records = {}'.format(sdfData2.count()))

Total Records = 193026


In [23]:
sdfData2.registerTempTable("validate")
output =  scSpark.sql('SELECT count(*) from validate')
output.show()

+--------+
|count(1)|
+--------+
|  193026|
+--------+



In [24]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:/opt/app-root/lib/python3.6/site-packages/pyspark/jars/mysql-connector-java-8.0.21.jar pyspark-shell'

output = scSpark.sql('SELECT * from validate')
output.show()
output.write.format('jdbc').options(
    url='jdbc:mysql://10.128.1.138/sepsis',
    #driver='com.mysql.jdbc.Driver',
    driver='com.mysql.cj.jdbc.Driver',
    #driver='mysql-connector-java.Driver',
    #driver='mysql-connector-python',
    dbtable='validate',
    user='root',
    password='Sepsis_Parkar_2020').mode('append').save()

+-------+----+-----+----+-----+-----+----+----+-----+----------+----+----+---+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+---+---+---+---+----------+---------+---+------+-----+-----+-----------+------+-----------+
|    PID|  HR|O2Sat|Temp|  SBP|  MAP| DBP|Resp|EtCO2|BaseExcess|HCO3|FiO2| pH|PaCO2|SaO2|AST|BUN|Alkalinephos|Calcium|Chloride|Creatinine|Bilirubin_direct|Glucose|Lactate|Magnesium|Phosphate|Potassium|Bilirubin_total|TroponinI|Hct|Hgb|PTT|WBC|Fibrinogen|Platelets|Age|Gender|Unit1|Unit2|HospAdmTime|ICULOS|SepsisLabel|
+-------+----+-----+----+-----+-----+----+----+-----+----------+----+----+---+-----+----+---+---+------------+-------+--------+----------+----------------+-------+-------+---------+---------+---------+---------------+---------+---+---+---+---+----------+---------+---+------+-----+-----+-----------+------+-----------+
|p110001| NaN|  NaN| NaN|  NaN|  NaN| NaN| 

Py4JJavaError: An error occurred while calling o159.save.
: com.mysql.cj.jdbc.exceptions.CommunicationsException: Communications link failure

The last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server.
	at com.mysql.cj.jdbc.exceptions.SQLError.createCommunicationsException(SQLError.java:174)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:64)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:836)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:456)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:246)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:197)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:61)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils